# Indice de relacion de estado economico sobre capitalizacion de mercado
## Introduccion y contexto
En las partes anteriores del proyecto se tenia como objetivo identificar que componentes eran mas importantes para detectar si una empresa esta o no cerca de la quiebra, para ello utilizamos un dataset en el que si una empresa era operativa o habia quebrado era un dato. En esta ocacion debido a que las empresas que extrajimos de la APIs no solo que aun estan operativas sino que ademas se encuentran en los indices mas importantes del mundo, esto hace que la posibilidad de bancarrota sea mucho menor (aunque nunca podra ser nula),es por esto que nos proponemos un objetivo diferente.

## Nuevo Objetivo
El nuevo objetivo sera descubrir las 10 empresas que se encuentran subvaluadas en comparacion a el total de las empresas del dataset que extrajimo de la API.

## Como lo lograremos
Nuestro dataset contiene 7 indices que en su conjunto nos indican el estado de salud financiero de la empresa, si tomamos en cuenta estos indices y los comparamos con la capitalizacion total de mercado de la empresa, obtendremos un nuevo indice que nos indicara que tan subvaluada o sobrevalorada esta nuestra empresa con respecto a las demas.
Cabe destacar que en este estudio no utilizaremos niveles de ventas, ya que los mismos solo reprecentan el tamaño de una empresa y no su estado financiero.

In [5]:
import pandas as pd
import requests
import json

In [6]:
# API Dataset
API_dataset = pd.read_csv('https://raw.githubusercontent.com/Gonzalo-Ariel-Alonso/Proyecto_Data_Science/master/API_dataset_cleaned', index_col = 0)

## Extracion de la capitalizacion de mercado

In [34]:
API_key = 'c54179160eae8b4c783528b88a551cf4'

In [8]:
for ticker in API_dataset.index:
    url = f"https://financialmodelingprep.com/api/v3/market-capitalization/{ticker}?apikey={API_key}"
    call_mk_cap = requests.get(url)
    
    if call_mk_cap.status_code == 200:
        
        mk_cap = call_mk_cap.json()[0]['marketCap']
        API_dataset.loc[ticker,'Market Cap.'] = mk_cap
        
    else: 
        print(f"Error al obtener el total assets. Código de estado: {call_mk_cap.status_code}")
        break
    

In [33]:
API_dataset

,Debt ratio %,Current Liability to Assets,Borrowing dependency (Debt To Equity),Current Liability to Current Assets,Net Income to Total Assets,Net worth/Assets,Persistent EPS in the Last Four Seasons (EPS),Working Capital/total assets,Market Cap.
AAPL,0.330007,0.059843,1.763490,0.155903,0.282924,0.143646,6.15,-0.020303,3.044290e+12
ABBV,0.461507,0.029797,4.677791,0.145311,0.085336,0.124304,6.65,-0.007939,2.507409e+11
ABNB,0.116695,0.003679,0.441504,0.003970,0.118032,0.346677,3.07,0.415077,9.405610e+10
ABT,0.225328,0.030240,0.225993,0.089240,0.093138,0.492840,3.91,0.130780,1.968752e+11
ACN,0.061766,0.015166,0.122223,0.033167,0.147873,0.467731,10.87,0.133032,1.986783e+11
...,...,...,...,...,...,...,...,...,...
WMT,0.260625,0.027122,0.882080,0.086979,0.046382,0.315017,4.29,-0.069803,4.285544e+11
XEL,0.427303,0.035644,1.548460,0.423989,0.028372,0.272521,3.17,-0.023011,3.588871e+10
XOM,0.112207,0.001718,0.208602,0.006494,0.156007,0.528492,13.26,0.082798,4.256045e+11
ZM,0.010676,0.002804,0.013968,0.003585,0.012760,0.763602,0.35,0.581170,2.075674e+10


## Estado economico

Para simplificar todos los indices que tenemos aremos una suma algebraica, donde aquellos indices que tienen un valor alto reprecenta una buena salud financiera tendran signo positivo y aquellos que un valor alto reprecente mala salud financiera tendran signo negativo. Para que esta cuenta no este sesgada por la dimension de valores, estos deben ser escalados.

In [11]:
# Escalado de datos
from sklearn.preprocessing import MinMaxScaler as MMS
scaler = MMS()
API_dataset_scaled = scaler.fit_transform(API_dataset)
API_dataset_scaled = pd.DataFrame(API_dataset_scaled, columns = list(API_dataset.columns), index = API_dataset.index )
API_dataset_scaled

,Debt ratio %,Current Liability to Assets,Borrowing dependency (Debt To Equity),Current Liability to Current Assets,Net Income to Total Assets,Net worth/Assets,Persistent EPS in the Last Four Seasons (EPS),Working Capital/total assets,Market Cap.
AAPL,0.327802,0.361560,0.646733,0.216566,0.917661,0.396472,0.298776,0.170780,1.000000
ABBV,0.462790,0.179966,1.000000,0.201851,0.515525,0.380429,0.308980,0.183143,0.082364
ABNB,0.108832,0.022110,0.486483,0.005480,0.582070,0.564869,0.235918,0.606103,0.030896
ABT,0.220347,0.182642,0.460359,0.123950,0.531405,0.686100,0.253061,0.321844,0.064670
ACN,0.052445,0.091534,0.447781,0.046045,0.642802,0.665274,0.395102,0.324095,0.065263
...,...,...,...,...,...,...,...,...,...
WMT,0.256580,0.163796,0.539889,0.120808,0.436246,0.538610,0.260816,0.121287,0.140773
XEL,0.427679,0.215305,0.620667,0.589029,0.399591,0.503363,0.237959,0.168073,0.011789
XOM,0.104225,0.010258,0.458251,0.008987,0.659357,0.715670,0.443878,0.273868,0.139804
ZM,0.000000,0.016822,0.434658,0.004946,0.367818,0.910675,0.180408,0.772174,0.006818


In [12]:
economic_status_index = {}
for company in API_dataset_scaled.index:
    economic_status_index[company] = {'Economic Status Index': sum(API_dataset_scaled.loc[company][4:8]) - sum(API_dataset_scaled.loc[company][0:4]),'Market Cap.': API_dataset_scaled.loc[company][8]}
df_economic_status_index  = pd.DataFrame.from_dict(economic_status_index,orient ='index')
df_economic_status_index

,Economic Status Index,Market Cap.
AAPL,0.231028,1.000000
ABBV,-0.456531,0.082364
ABNB,1.366055,0.030896
ABT,0.805112,0.064670
ACN,1.389468,0.065263
...,...,...
WMT,0.275886,0.140773
XEL,-0.543694,0.011789
XOM,1.511052,0.139804
ZM,1.774650,0.006818


Ahora que juntamos todos los indices en una sola columna, podemos emprezar a filtrar algunas empresas: ya que nuestro nuevo indice es la sumatoria algebraica de nuestros anteriores indices, podemos deducir que a mayor valor de nuestro indice, mayor salud economica. Por ende, podemos descartar las empresas con indice negativo, ya que incluso teniendo una capitalizacion baja, si los numeros financieron no acopañan no tiene sentido invertir en la misma. 

In [17]:
df_economic_status_index = df_economic_status_index[df_economic_status_index['Economic Status Index'] > 0]
df_economic_status_index

,Economic Status Index,Market Cap.
AAPL,0.231028,1.000000
ABNB,1.366055,0.030896
ABT,0.805112,0.064670
ACN,1.389468,0.065263
ADBE,1.193418,0.078024
...,...,...
WFC,0.129083,0.055293
WMT,0.275886,0.140773
XOM,1.511052,0.139804
ZM,1.774650,0.006818


## Indice de relacion de estado economico sobre capitalizacion de mercado
La ultima cuenta que aremos sera dividir nuestro indice de salud financiera sobre la capitalizacion de mercado

In [18]:
ES_divided_MC = df_economic_status_index['Economic Status Index'] / df_economic_status_index['Market Cap.']

In [28]:
df_economic_status_index['Economic Status Index / Market Cap'] = ES_divided_MC

C:\Users\gonar\AppData\Local\Temp\ipykernel_3252\2946551203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_economic_status_index['Economic Status Index / Market Cap'] = ES_divided_MC


In [29]:
df_economic_status_index

,Economic Status Index,Market Cap.,Economic Status Index / Market Cap
AAPL,0.231028,1.000000,0.231028
ABNB,1.366055,0.030896,44.214765
ABT,0.805112,0.064670,12.449474
ACN,1.389468,0.065263,21.290406
ADBE,1.193418,0.078024,15.295596
...,...,...,...
WFC,0.129083,0.055293,2.334527
WMT,0.275886,0.140773,1.959791
XOM,1.511052,0.139804,10.808345
ZM,1.774650,0.006818,260.279259


Ordenamos de mayor a menor por nuestro nuevo indice y filtramos las 10 empresas que mejor rendimiento tiene en relacion al estado economico comparado con su capitalizacion de mercado

In [32]:
df_economic_status_index.sort_values(by = 'Economic Status Index / Market Cap',ascending = False).head(10)

,Economic Status Index,Market Cap.,Economic Status Index / Market Cap
ZM,1.774650,0.006818,260.279259
FANG,1.539649,0.008653,177.922284
MRNA,2.480173,0.015402,161.027535
CPRT,2.156746,0.014061,153.379570
ALGN,1.268197,0.008290,152.981159
CSGP,1.743996,0.012279,142.028931
ODFL,2.006299,0.014244,140.850065
FAST,1.478368,0.010998,134.425510
CTSH,1.507574,0.011264,133.842590
TRV,1.715707,0.013128,130.692733


## Resultados
Las empresas con mejor salud financiera y menor capitalizacion de mercado son: 
- ZM (Zoom Video Communications)
- FANG (Diamondback Energy Inc)
- MRNA (Moderna)
- CORT (Copart)
- ALGN (Align Technology)
- CSGP (CoStar Group Inc)
- ODFL (Old Dominion Freight Line Inc)
- FAST (Fastenal)
- CTSH (Cognizant)
- TRV (The Travelers Companies)
